PLAN:
* HUMAN data:
   - adapt analysis scripts from NNDL folder (create functions and call them);
   - check that all info can be retrieved from output files

* MACHINES data:
   - report functions, should be called on raw data from experiments

* COMBINED analysis:
   - create and save plot for comparing human and machines results (with problem phases)